<img src="https://raw.githubusercontent.com/dask/dask/main/docs/source/images/dask_horizontal.svg"
     width="60%"
     alt="Dask logo\" />

# Time for a Test Drive!

You've spent some time walking around the Dascar lot, hearing about all the awesome features and specs...

That's enough talk. Let's jump into this racecar and see what it can do...

We'll test drive:

1. Dask DataFrames for faster & scalable pandas
2. Dask Arrays for faster & scalable NumPy

![](images/race-car.png "Title")

## Dask DataFrames

The pandas car...with the Dask engine!

In [ ]:
import dask.dataframe as dd

In [ ]:
%run ../prep_data.py -d flights

In [ ]:
import os

files = os.path.join('../data', 'nycflights', '*.csv')
files

In [ ]:
df = dd.read_csv(files,
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={"TailNum": str,
                        "CRSElapsedTime": float,
                        "Cancelled": bool})

In [ ]:
df.head()

In [ ]:
%%time
df.groupby("Origin")["DepDelay"].mean().compute()

### A slight difference with pandas
Notice the `.compute()` call: this is necessary because Dask operates using something called **lazy evaluation**.

If you haven't heard about lazy evaluation before, check out [the Beginner's Guide to Distributed Computing](https://towardsdatascience.com/the-beginners-guide-to-distributed-computing-6d6833796318).

In [ ]:
df

In [ ]:
df.visualize()

## Dask Arrays

The Numpy car...with Dask engine superpowers!

In [ ]:
import dask.array as da

In [ ]:
array = da.random.random((10_000, 10_000), chunks=(1_000, 1_000))

In [ ]:
array

In [ ]:
array[:10,:5]

In [ ]:
array[:10,:5].compute()

In [ ]:
%%time
array.sum(axis=1).compute()

# Digging Deeper

Dask's lower-level APIs give you even more flexibility and control over what / how to parallelize your custom Python code.

## Parallelize Python Code with `dask.delayed`

In [ ]:
from time import sleep

def inc(x):
    """Increments x by one"""
    sleep(1)
    return x + 1

def add(x=0, y=0, z=0):
    """Adds x and y and z"""
    sleep(1)
    return x + y + z

In [ ]:
%%time

x = inc(1) # takes 1 second
y = inc(2) # takes 1 second
z = add(x, y) # takes 1 second

In [ ]:
z

In [ ]:
from dask import delayed

In [ ]:
%%time

a = delayed(inc)(1)
b = delayed(inc)(2)
c = delayed(add)(a, b)

In [ ]:
c

In [ ]:
a.visualize()

In [ ]:
b.visualize()

In [ ]:
c.visualize()

In [ ]:
%%time
c.compute()

In [ ]:
d = delayed(inc)(3)

In [ ]:
c = delayed(add)(a, b, d)

In [ ]:
c.visualize()

In [ ]:
%%time
c.compute()

Task graphs can get...complicated:

<img src="https://raw.githubusercontent.com/coiled/pydata-global-dask/master/images/grid_search_schedule.gif"
     width="95%"
     alt="Grid search schedule\" />